In [1]:
import mne
import glob
import os
import numpy as np
mne.set_log_level("error")


%matplotlib qt

In [ ]:
import os
import glob
import numpy as np
import mne
import pandas as pd

paths_byjhana = glob.glob('../../eeg_jhana/notebooks/hand_cleaning/resting_state_byjhana_selected/*')
paths_full = glob.glob('../../eeg_jhana/notebooks/hand_cleaning/resting_state/*')

log = []  # For Excel output

for paths in [paths_full, paths_byjhana]:

    paths.sort()

    for path_i, path in enumerate(paths): 
        raw = mne.io.read_raw(path, preload=True)
        num_bad_channels = len(raw.info['bads'])  # Track before interpolation

        basename = os.path.basename(path)
        sub = basename[:4]
        day = basename[5:9]
        condition = basename[10:-8]
        if len(basename) == 36:
            condition = basename[10:15] + basename[-5:-4]

        raw.interpolate_bads(reset_bads=True)

        bad_segments = []
        for annot in raw.annotations:
            if annot['description'].lower() == 'bad_':
                bad_segments.append((annot['onset']-raw.first_time, annot['onset'] + annot['duration']-raw.first_time))

        # Get all good segments
        good_segments = []
        start_time = 0  # Start from the beginning

        for bad_start, bad_end in bad_segments:
            if bad_start - start_time >= 10:
                good_segments.append((start_time, bad_start))
            start_time = bad_end  # Move start time to after the bad segment

        # Handle last segment if it exists
        if start_time < raw.times[-1]:  
            good_segments.append((start_time, raw.times[-1]))

        # Extract and concatenate the valid good segments
        good_raws = [raw.copy().crop(tmin, tmax) for tmin, tmax in good_segments]
        if len(good_raws) > 0:
            raw_cleaned = mne.concatenate_raws(good_raws)
            
            print (f'epoch is {np.round(len(raw_cleaned)/256/60, 2)} minutes')
            
            export_name = os.path.basename(path)
        
            raw_cleaned.save('input/continuous/' + os.path.basename(path), overwrite=True)

            # Log cleaned duration
            duration_minutes = np.round(len(raw_cleaned) / 256 / 60, 2)
        else:
            duration_minutes = 0

        try: 
            epochs = mne.make_fixed_length_epochs(raw, duration=10, preload=True)
            if len(epochs)>0:
                epochs.save('input/10s/' + os.path.basename(path), overwrite=True)
        except:
            print(f"{os.path.basename(path)} could not generate epochs")
            epochs = []            

        # Add entry to log
        log.append({
            'filename': basename,
            'sub': sub,
            'day': day,
            'condition': condition,
            'duration_minutes': duration_minutes,
            'num_epochs_10s': len(epochs),
            'num_bad_channels': num_bad_channels
        })

# Save Excel sheet
df = pd.DataFrame(log)
df.to_excel('segment_summary.xlsx', index=False)
print("✅ Excel summary saved to segment_summary.xlsx")


KeyboardInterrupt: 